In [1]:
import xml.etree.ElementTree as ET
import io
import pandas as pd
import re
import numpy as np
import warnings

In [2]:
xml_tree = ET.parse("Zoo_ERM.drawio.xml")
# xml_tree = ET.parse("ETL.drawio.xml")
# xml_tree = ET.parse("Baustoffgroßhandel.drawio.xml")
# xml_tree = ET.parse('Stadtbibliothek.drawio.xml')
# xml_tree = ET.parse("KFZ_Vermietung.drawio.xml")
# xml_tree = ET.parse('Test.drawio.xml')
xml_root = xml_tree.getroot()[0][0][0]

In [3]:
class Node:
    def __init__(self, nodeId, data, level = None):
        self.nodeId = nodeId
        self.data = data
        self.level = level
        self.children = []
    def __str__(self):
        return str(self.data)

class Tree:
    def __init__(self):
        self.root = None
    def addNode(self, parentId, nodeId, data):
        if self.root == None:
            self.root = Node(nodeId, data, 0)
        else:
            # breath first search
            q = []
            q.append(self.root)
            while q:
                temp = q.pop()
                if temp.nodeId == parentId:
                    temp.children.append(Node(nodeId, data, temp.level+1))
                    break
                else:
                    q += temp.children

                

In [4]:
def printTree(node):
    if not node:
        return
    for i in range(node.level):
        print("X  ", end = "")
    print(node.data)
    for i in node.children:
        printTree(i)

# tempTaT: String
tempTaT = ""

def printTreeAsTable(node):
    global out
    if not node:
        return
    global tempTaT
    if node.level == 2:
        tempTaT = node.data
    elif node.level == 3:
        print(f"\n{tempTaT} ", end = "")
    elif node.level == 4:
        print(f"{node.data} ", end = "")
        # print(f"{tempTaT} {node.data}", end = "")
    for i in node.children:
        printTreeAsTable(i) 

def writeTreeAsCSV_helper(node):
    if not node:
        return
    global tempTaT
    if node.level == 2:
        tempTaT = node.data
    elif node.level == 3:
        out.write(f"\n\"{tempTaT}\";")
    elif node.level == 4:
        out.write(f"\"{node.data}\";")
    for i in node.children:
        writeTreeAsCSV_helper(i)

# out: fileIO
out = None
def writeTreeAsCSV(node):
    global out
    out = io.open("TableOutputFromDrawIO.csv", "w")
    writeTreeAsCSV_helper(node)
    out.close()

In [5]:
tree = Tree()
for child in xml_root:
    ## get rid of any <...> statements
    temp = re.sub(r"<.*?>", "", (child.attrib).get("value", ""))
    tree.addNode((child.attrib).get("parent", ""), (child.attrib).get("id", ""), temp)
#    tree.addNode((child.attrib).get("parent", ""), (child.attrib).get("id", ""), (child.attrib).get("value", ""))

In [6]:
writeTreeAsCSV(tree.root)

In [7]:
table = pd.read_csv("TableOutputFromDrawIO.csv",sep = ';', header = None)
table.rename(columns = {0: "Tabelle", 1: "PK", 2: "Feldname", 3: "Beschreibung"}, inplace = True)
table.insert(2,"FK", table.loc[:,"PK"])
table["PK"] = table["PK"].map(lambda x: False if pd.isnull(x) else bool(re.match(r".*PK.*", x)))
table["FK"] = table["FK"].map(lambda x: False if pd.isnull(x) else bool(re.match(r".*FK.*", x)))

In [8]:
table.loc[:,"Felddatentyp"] = ["INTEGER" if x else "TEXT" for x in (table["PK"] | table["FK"])]
table.loc[:, "Laenge_Zeichen"] = ["" if x else "50" for x in (table["PK"] | table["FK"])]
table.loc[:, "NN"] = table["PK"] | table["FK"]
table.loc[:,"AI"] = table["PK"] & ~ table["FK"]
table.loc[:,"Referenz"] = ""

table = table.loc[:, ["Tabelle", "Feldname", "Felddatentyp", "Laenge_Zeichen", "PK", "FK", "Referenz", "AI", "NN", "Beschreibung"]]

In [9]:
# CAVE!!!
# Automatic assignment of the References.
# The Foreign Key HAVE to match the Primary Key
# The Primary Keys HAVE to be unique

## FK with the same name in the entity are coded as PK_(specification)
def extractPKfromFK (FK):
    if FK is np.nan:
        return FK
    temp = re.search(r"^(\w+)_\(", FK)

    if temp is not None:
        return temp.group(0)[:-2]
    else:
        return FK

## added error handling with try except
temp_PK = table.loc[table.loc[:,"PK"] & ~ table.loc[:,"FK"] , :]
temp_ref = []
for i in table.loc[table.loc[:,"FK"], "Feldname"]:
    try:
        temp_ref.append(temp_PK.loc[temp_PK.loc[:, "Feldname"] == extractPKfromFK(i), "Tabelle"].array[0])
    except IndexError:
        warnings.warn(f"{i} can not be matched to any primary keys")
        temp_ref.append("Missing_Ref")
table.loc[table.loc[:,"FK"], "Referenz"] = temp_ref

    

## old version without error handling
# temp_PK = table.loc[table.loc[:,"PK"] & ~ table.loc[:,"FK"] , :]
# table.loc[table.loc[:,"FK"], "Referenz"] = [temp_PK.loc[temp_PK.loc[:, "Feldname"] == i, "Tabelle"].array[0] for i in table.loc[table.loc[:,"FK"], "Feldname"]]


In [10]:
## To debug foreign - primary key missmatches
# for i in table.loc[table.loc[:,"FK"], "Feldname"]:
#     print(temp_PK.loc[temp_PK.loc[:, "Feldname"] == i, "Tabelle"].array[0] )
# table.loc[table.loc[:,"FK"], "Feldname"]
# temp_PK

In [11]:
table.to_csv("DataDictionaryFromDrawIO.csv", index = False)

In [12]:
## Testing
# printTreeAsTable(tree.root)
# printTree(tree.root)
# for child in root[0:10]:
#     print(child.attrib)
# for child in root[0:10]:
#     print(f"id: {(child.attrib).get("id", "")}; value: {(child.attrib).get("value", "")}, parent: {(child.attrib).get("parent", "")}")
